#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   (quando tiver)


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}

In [ ]:
import pandas as pd

df = pd.read_excel('../produtos.xlsx')

comissao_percentual = 0.185
taxa_transacao_percentual = 0.01
taxa_fixa_padrao = 4
margem_lucro_desejada = 0.20

def calcular_preco_venda(preco_custo):

    if preco_custo <= 8:
        taxa_aplicada = preco_custo / 2
    else:
        taxa_aplicada = taxa_fixa_padrao

    total_percentuais = comissao_percentual + taxa_transacao_percentual + margem_lucro_desejada

    preco_venda = (preco_custo + taxa_aplicada) / (1 - total_percentuais)

    comissao = preco_venda * comissao_percentual
    taxa_transacao = preco_venda * taxa_transacao_percentual
    total_taxas = comissao + taxa_aplicada + taxa_transacao

    lucro = preco_venda - preco_custo - total_taxas

    return pd.Series({
        'Preço Final': round(preco_venda, 2),
        'Taxas': round(total_taxas, 2),
        'Lucro': round(lucro, 2)
    })

produtos_completos = df.copy()
produtos_completos.loc[:, ['Preço Final', 'Taxas', 'Lucro']] = (
    produtos_completos['Valor_unitário'].apply(calcular_preco_venda)
)

produtos_completos

In [ ]:
import pandas as pd

df = pd.read_excel('/home/lucas-silva/auto_shopee/planilhas/outputs/final.xlsx')

marcas = df['Marca'].unique().tolist()

marcas

In [ ]:
import re
import pandas as pd

# Certifique-se que o nome da coluna é exatamente este:
col = "Descrição"

df = pd.read_excel('/home/lucas-silva/github/auto_shopee/planilhas/outputs/final.xlsx')

# Lista de categorias pesadas
heavy_keywords = [
    r"\bTELHA\b", r"\bCUMEEIRA\b",
    r"\bPORTA\b", r"GABINETE", r"\bVASO\b", r"LAVAT[ÓO]RIO", r"\bPIA\b", r"\bTANQUE\b",
    r"CAIXA D'?[ÁA]GUA",r"CANTON", r"EXTENSOR", r"BACIA",r"VIAPLUS", r"VEDAPREN",
    r"TUBO", r"CONEX[ÃA]O",r"CANAL", r"LONA", r"TECPLUS",r"SIKATOP", r"CORRENTE",
    r"CIMENTO", r"ARGAMASSA", r"MASSA CORRIDA", r"MASSA ACR[IÍ]LICA",
    r"TINTA", r"VEDATOP", r"\bCALHA\b",r"ELETRODUTO",r"VAR[ÃA]O", r"REGUA", r"RUFO",
    r"\bPÁ\b", r"\bENXADA\b", r"CAVADEIRA", r"\bFORCADO\b", r"R[EÉ]GUA ALUM[IÍ]NIO",
    r"CARRINHO", r"\bESCADA\b", r"VARAL DE CH[ÃA]O",r"MANTA", r"DAGUA",r"COLUNA", r"\bLAV\b",r"TELA"
]

# Falsos positivos que devem ser ignorados mesmo contendo palavras pesadas
exclude_keywords = [
    "PARAFUSO P/ TELHA", "PARAF","FECH","SOLDA","LIGACAO","VALV","ANEL","SERRA","BOCAL",
    "ABRAC", "CADEADO ", "GRELHA", "TORN","DESEMP","RALINHO","PINO", "BOLSA","ADAP","PLASTICO",
    "TRAVA", "BOLSA","LIMA", "RALINHO","FITA","CABO FLEX", "CABO REDE", "PATCH CORD", "COAXIAL",
    "CABO PP","BATEDOR","BALDE","ABRAC","SELATRINCA","FIXADOR","PILHA","FRISO","MOVEIS","REFIL",
    "VIDRO", "PRATELEIRA","FRANC", "PRESSURIZADOR", "FILTRO","PENEIRA","LUVA","CALCO","DISCO",
    "\bSUP\b", "BATEADOR","MISTURADOR","SAIDA", "CABECEIRA", "PNEU P/ CARRINHO", "SUPORTE"
]

pesados_dict = {}

for idx, desc in df[col].items():
    d = str(desc).upper()

    # Exclusões diretas
    if any(ex in d for ex in exclude_keywords):
        continue

        # Massa corrida leve (não entra)
    if ("MASSA CORRIDA" in d or "MASSA ACR" in d) and any(x in d for x in ["1KG", "900G", "500G", "3.6KG"]):
        continue

    # Tinta só é pesada se 18L ou + (seu padrão)
    if "TINTA" in d and not any(x in d for x in ["15","18L", "20L"]):
        # continue

    # Checagem principal
    if any(re.search(k, d) for k in heavy_keywords):
        pesados_dict[idx] = desc

# Criar dataframes finais
df_pesados = df.loc[df.index.isin(pesados_dict.keys())].copy()
df_restante = df.loc[~df.index.isin(pesados_dict.keys())].copy()


print("Itens pesados:", len(df_pesados))
print("Itens restantes:", len(df_restante))

df_restante.to_excel('grandes.xlsx')

df_pesados

Itens pesados: 340
Itens restantes: 3761


,Codigo Produto,Descrição,Valor_unitário,Código de Barras,Sku,Fornecedor,Data Emissão,NCM,Marca,Peso,...,GTIN_Válido,Preço Final,Taxas,Lucro,Chave,ID_Variacao,Tipo,SKU_Pai,Base,Variante
114,16436,TINTA PO VERDE 5KG SAMPAFLEX,10.360,7898958256272,07898958256272,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2023-07-26 05:34:00,25222000,SAMPAFLEX,5.00,...,True,20.51,8.10,2.05,TINTA PO VERDE 5KG SAMPAFLEX,55,PAI,07898958256272,TINTA PO VERDE 5KG SAMPAFLEX,NaN
131,190824,CANTON ALUM L PRETA (C) 3/4 FERJA,14.290,7899932009372,07899932009372,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2021-11-27 00:53:00,76042920,FERJA,0.16,...,True,26.13,9.23,2.61,CANTON ALUM L PRETA C 34 FERJA,65,PAI,07899932009372,CANTON ALUM L PRETA C FERJA,CANTON ALUM L PRETA C 34 FERJA
185,16425,TINTA PO AMARELO 5KG SAMPAFLEX,10.360,17898958256156,17898958256156,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2023-07-26 05:34:00,25222000,SAMPAFLEX,5.00,...,True,20.51,8.10,2.05,TINTA PO AMARELO 5KG SAMPAFLEX,82,PAI,17898958256156,TINTA PO AMARELO 5KG SAMPAFLEX,NaN
199,184199,"TELA NYLON 1,20X50M CZ* TECNYL",139.044,618231087419,618231087419,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2024-12-07 02:07:00,56031210,TECNYL,3.01,...,True,204.35,44.87,20.43,TELA NYLON 120X50M CZ TECNYL,87,PAI,618231087419,TELA NYLON 120X50M CZ TECNYL,NaN
206,10103,TELHA 244X050X4 S/A MULTILIT,18.389,7890000101031,TELHA 244X050X4 S/A,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2024-12-31 02:23:00,68118100,MULTILIT,9.29,...,True,31.98,10.40,3.20,TELHA 244X050X4 SA MULTILIT,92,PAI,TELHA 244X050X4 S/A,TELHA 244X050X4 SA MULTILIT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,83585,ELETRODUTO ROSC PVC A 1/2,6.741,1027677359254,ELETRODUTO ROSC PVC A 1/2,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2021-10-09 02:40:52,39172300,GENÉRICO,1.00,...,True,14.45,6.26,1.44,ELETRODUTO ROSC PVC A 12,2777,PAI,ELETRODUTO ROSC PVC A 1/2,ELETRODUTO ROSC PVC A,12
4082,83593,ELETRODUTO ROSC PVC B 3/4,7.922,5562588153714,ELETRODUTO ROSC PVC B 3/4,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2021-10-09 02:40:52,39172300,GENÉRICO,1.00,...,True,16.98,7.36,1.70,ELETRODUTO ROSC PVC B 34,2778,PAI,ELETRODUTO ROSC PVC B 3/4,ELETRODUTO ROSC PVC B,34
4091,237400,CUMEEIRA UNIVERSAL 110 S/AMIAN BRASILIT,24.880,3097289623096,CUMEEIRA UNIVERSAL 110 S/AMIAN BRASILIT,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2020-10-27 03:57:42,68118200,BRASILIT,1.00,...,True,41.26,12.25,4.13,CUMEEIRA UNIVERSAL 110 SAMIAN BRASILIT,2784,PAI,CUMEEIRA UNIVERSAL 110 S/AMIAN BRASILIT,CUMEEIRA UNIVERSAL SAMIAN BRASILIT,CUMEEIRA UNIVERSAL 110 SAMIAN BRASILIT
4099,279323,TUBO CORR PLASTIK ESGOTO 50MM BARRA 6MT,26.590,9187851910885,TUBO CORR PLASTIK ESGOTO 50MM BARRA 6MT,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2020-01-21 04:30:21,39172300,PLASTIK,1.00,...,True,43.70,12.74,4.37,TUBO CORR PLASTIK ESGOTO 50MM BARRA 6MT,2792,FILHO,TUBO CORR PLASTIK ESGOTO 100MM BARRA 6MT,TUBO CORR PLASTIK ESGOTO 50MM BARRA 6MT,NaN
